# PUBG Finish Placement Prediction

A model which predicts players' finishing placement based on their final stats, on a scale from 1 (first place) to 0 (last place). 

<img src="Images/1.jpg" alt="drawing" width="800px"/>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("train_V2.csv")
df.columns = ['Id','groupId','matchId','assists','boosts','damageDealt','DBNOs','headshotKills','heals','killPlace','killPoints','kills','killStreaks','longestKill','matchDuration','matchType','maxPlace','numGroups','rankPoints','revives','rideDistance','roadKills','swimDistance','teamKills','vehicleDestroys','walkDistance','weaponsAcquired','winPoints','winPlacePerc']

In [3]:
df.describe()

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
count,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,...,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446965e+06
mean,2.338149e-01,1.106908e+00,1.307171e+02,6.578755e-01,2.268196e-01,1.370147e+00,4.759935e+01,5.050060e+02,9.247833e-01,5.439551e-01,...,1.646590e-01,6.061157e+02,3.496091e-03,4.509322e+00,2.386841e-02,7.918208e-03,1.154218e+03,3.660488e+00,6.064601e+02,4.728216e-01
std,5.885731e-01,1.715794e+00,1.707806e+02,1.145743e+00,6.021553e-01,2.679982e+00,2.746294e+01,6.275049e+02,1.558445e+00,7.109721e-01,...,4.721671e-01,1.498344e+03,7.337297e-02,3.050220e+01,1.673935e-01,9.261157e-02,1.183497e+03,2.456544e+00,7.397004e+02,3.074050e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.551000e+02,2.000000e+00,0.000000e+00,2.000000e-01
50%,0.000000e+00,0.000000e+00,8.424000e+01,0.000000e+00,0.000000e+00,0.000000e+00,4.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.856000e+02,3.000000e+00,0.000000e+00,4.583000e-01
75%,0.000000e+00,2.000000e+00,1.860000e+02,1.000000e+00,0.000000e+00,2.000000e+00,7.100000e+01,1.172000e+03,1.000000e+00,1.000000e+00,...,0.000000e+00,1.909750e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.976000e+03,5.000000e+00,1.495000e+03,7.407000e-01
max,2.200000e+01,3.300000e+01,6.616000e+03,5.300000e+01,6.400000e+01,8.000000e+01,1.010000e+02,2.170000e+03,7.200000e+01,2.000000e+01,...,3.900000e+01,4.071000e+04,1.800000e+01,3.823000e+03,1.200000e+01,5.000000e+00,2.578000e+04,2.360000e+02,2.013000e+03,1.000000e+00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446966 entries, 0 to 4446965
Data columns (total 29 columns):
Id                 object
groupId            object
matchId            object
assists            int64
boosts             int64
damageDealt        float64
DBNOs              int64
headshotKills      int64
heals              int64
killPlace          int64
killPoints         int64
kills              int64
killStreaks        int64
longestKill        float64
matchDuration      int64
matchType          object
maxPlace           int64
numGroups          int64
rankPoints         int64
revives            int64
rideDistance       float64
roadKills          int64
swimDistance       float64
teamKills          int64
vehicleDestroys    int64
walkDistance       float64
weaponsAcquired    int64
winPoints          int64
winPlacePerc       float64
dtypes: float64(6), int64(19), object(4)
memory usage: 983.9+ MB


## A description of the attributes

- groupId - Integer ID to identify a group within a match. If the same group of players plays in different matches, they will have a different groupId each time.
- matchId - Integer ID to identify match. There are no matches that are in both the training and testing set.
- assists - Number of enemy players this player damaged that were killed by teammates.
- boosts - Number of boost items used.
- damageDealt - Total damage dealt. Note: Self inflicted damage is subtracted.
- DBNOs - Number of enemy players knocked.
- headshotKills - Number of enemy players killed with headshots.
- heals - Number of healing items used.
- killPlace - Ranking in match of number of enemy players killed.
- killPoints - Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.)
- kills - Number of enemy players killed.
- killStreaks - Max number of enemy players killed in a short amount of time.
- longestKill - Longest distance between player and player killed at time of death. This may be misleading, as downing a - player and driving away may lead to a large longestKill stat.
- maxPlace - Worst placement we have data for in the match. This may not match with numGroups, as sometimes the data skips over placements.
- numGroups - Number of groups we have data for in the match.
- revives - Number of times this player revived teammates.
- rideDistance - Total distance traveled in vehicles measured in meters.
- roadKills - Number of kills while in a vehicle.
- swimDistance - Total distance traveled by swimming measured in meters.
- teamKills - Number of times this player killed a teammate.
- vehicleDestroys - Number of vehicles destroyed.
- walkDistance - Total distance traveled on foot measured in meters.
- weaponsAcquired - Number of weapons picked up.
- winPoints - Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.)
- winPlacePerc - The target of prediction. This is a percentile winning placement, where 1 corresponds to 1st place, and 0 corresponds to last place in the match. It is calculated off of maxPlace, not numGroups, so it is possible to have missing chunks in a match.



# Preprocessing

## Illegal Match

In [5]:
df.isnull().sum()

Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       1
dtype: int64

As it can be seen that the attribute 'winPlacePrec' has a single null value.

In [6]:
index = df[df['winPlacePerc'].isnull()].index.tolist()
df.loc[index, 'numGroups']

2744604    1
Name: numGroups, dtype: int64

It can be seen that this match has only one player. This can be thought of an illegal match and hence we remove this row.

<img src="Images/2.png" alt="drawing" width="800px"/>

In [7]:
df.drop(index, inplace=True)
df.isnull().sum()

Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       0
dtype: int64

## Kills without Moving, Extreme number of Kills and Headshot Kills

<img src="Images/9.jpg" alt="drawing" width="800px"/>

In [8]:
df['totaldistance'] = df['rideDistance'] + df['swimDistance'] + df['walkDistance']
df['killswithoutmoving'] = ((df['totaldistance']==0) & (df['kills'] > 0))
print(df[df['killswithoutmoving'] == True].shape)
df.drop(df[df['killswithoutmoving'] == True].index, inplace=True)

(1535, 31)


In [9]:
print(df[df['kills'] > 40].shape)
df.drop(df[df['kills'] > 40].index, inplace=True)

(32, 31)


In [10]:
print(df[(df['headshotKills']/df['kills'] == 1) & (df['kills'] > 12)].shape)
df.drop(df[(df['headshotKills']/df['kills'] == 1) & (df['kills'] > 12)].index, inplace=True)

(6, 31)


## Only swimming and still winning

<img src="Images/10.jpg" alt="drawing" width="800px"/>

In [11]:
df['totaldistance'] = df['rideDistance'] + df['swimDistance'] + df['walkDistance']
print(df[(df['totaldistance'] == df['swimDistance'])&(df['winPlacePerc'] > 0.80)].shape)
df.drop(df[(df['totaldistance'] == df['swimDistance'])&(df['winPlacePerc'] > 0.80)].index, inplace=True)

(991, 31)


## Excessive Weapons Acquired

<img src="Images/11.png" alt="drawing" width="800px"/>

In [12]:
print(df[df['weaponsAcquired'] > 50].shape)
df.drop(df[df['weaponsAcquired'] > 50].index, inplace=True)

(133, 31)


## Excessive Revivals

<img src="Images/12.jpg" alt="drawing" width="800px"/>

In [13]:
print(df[df['heals'] > 35].shape)
df.drop(df[df['heals'] > 35].index, inplace=True)

(273, 31)


## A general outlier detection

In [14]:
print(df[(df['weaponsAcquired'] > 35) & (df['totaldistance'] == 0)].shape)
df.drop(df[(df['weaponsAcquired'] > 35) & (df['totaldistance'] == 0)].index, inplace=True)

(1, 31)


In [15]:
print(df[(df['weaponsAcquired'] > 30) & (df['totaldistance'] < 50) & (df.kills > 30)].shape)
df.drop(df[(df['weaponsAcquired'] > 30) & (df['totaldistance'] < 50) & (df.kills > 30)].index, inplace=True)

(2, 31)


In [16]:
print(df[df.longestKill > 950].shape)
df.drop(df[df['longestKill'] > 950].index, inplace=True)

(84, 31)


# Exploratory Data Analysis

### Kills, Players knocked down and Assists

<img src="Images/6.jpg" alt="drawing" width="800px"/>

In [ ]:
data = df.copy()
data.loc[data['kills'] > data['kills'].quantile(0.99)] = '8+'
plt.figure(figsize=(15,10))
sns.countplot(data['kills'].astype('str').sort_values())
plt.title("Kill Count",fontsize=15)
plt.show()

In [ ]:
kills = df.copy()

kills['killsCategories'] = pd.cut(kills['kills'], [-1, 0, 2, 5, 10, 60], labels=['0_kills','1-2_kills', '3-5_kills', '6-10_kills', '10+_kills'])

plt.figure(figsize=(15,10))
sns.boxplot(x="killsCategories", y="winPlacePerc", data=kills)
plt.show()

In [ ]:
data = df.copy()
data.loc[data['DBNOs'] > data['DBNOs'].quantile(0.99)] = '8+'
plt.figure(figsize=(15,10))
sns.countplot(data['DBNOs'].astype('str').sort_values())
plt.title("Knocked Players Count",fontsize=15)
plt.show()

In [ ]:
knocked = df.copy()

knocked['knockedCategories'] = pd.cut(knocked['DBNOs'], [-1, 0, 2, 5, 10, 60], labels=['0_knocks','1-2_knocks', '3-5_knocks', '6-10_knocks', '10+_knocks'])

plt.figure(figsize=(15,8))
sns.boxplot(x="knockedCategories", y="winPlacePerc", data=knocked)
plt.show()

In [ ]:
data = df.copy()
data.loc[data['assists'] > data['assists'].quantile(0.99)] = '4+'
plt.figure(figsize=(15,10))
sns.countplot(data['assists'].astype('str').sort_values())
plt.title("Assists Count",fontsize=15)
plt.show()

In [ ]:
assists = df.copy()

assists['assistsCategories'] = pd.cut(assists['assists'], [-1, 0, 2, 5, 10], labels=['0_assists','1-2_assists', '3-5_assists', '5+_assists'])

plt.figure(figsize=(15,8))
sns.boxplot(x="assistsCategories", y="winPlacePerc", data=assists)
plt.show()

## Distance Travelled : Walk distance, Ride distance

<img src="Images/4.png" alt="drawing" width="800px"/>

In [ ]:
x = df.walkDistance
sns.distplot(x)

In [ ]:
plot = df.plot(x='walkDistance', y='winPlacePerc', style='.')
y_label = plot.set_ylabel('winPlacePerc')

In [ ]:
x = df.rideDistance
sns.distplot(x)

In [ ]:
plot = df.plot(x='rideDistance', y='winPlacePerc', style='.')
y_label = plot.set_ylabel('winPlacePerc')

### Heals and Boosts

<img src="Images/5.jpg" alt="drawing" width="800px"/>

In [ ]:
data = df.copy()

f,ax1 = plt.subplots(figsize =(20,10))
sns.pointplot(x='heals',y='winPlacePerc',data=data,color='green',alpha=0.8)
sns.pointplot(x='boosts',y='winPlacePerc',data=data,color='blue',alpha=0.8)
plt.text(4,0.6,'Heals',color='lime',fontsize = 17)
plt.text(4,0.55,'Boosts',color='blue',fontsize = 17)
plt.xlabel('Number of heal/boost items',fontsize = 15)
plt.ylabel('Win Percentage',fontsize = 15)
plt.title('Heals vs Boosts',fontsize = 15)
plt.grid()
plt.show()

## Weapons Acquired

<img src="Images/7.jpg" alt="drawing" width="800px"/>

In [ ]:
x = df.weaponsAcquired
sns.distplot(x)

In [ ]:
plot = df.plot(x='weaponsAcquired', y='winPlacePerc', style='.')
y_label = plot.set_ylabel('winPlacePerc')

## Correlation Matrix 

<img src="Images/8.png" alt="drawing" width="800px"/>

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
k = 7
f,ax = plt.subplots(figsize=(11, 11))
cols = df.corr().nlargest(k, 'winPlacePerc')['winPlacePerc'].index
cm = np.corrcoef(df[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values, cmap='coolwarm')
plt.show()

# Feature Engineering

### Creating New Features

In [17]:
df['playersJoined'] = df.groupby('matchId')['matchId'].transform('count')
df['healsandboosts'] = df['heals'] + df['boosts']
# df['totaldistance'] = df['rideDistance'] + df['swimDistance'] + df['walkDistance']
df.describe()

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,...,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc,totaldistance,playersJoined,healsandboosts
count,4.443908e+06,4.443908e+06,4.443908e+06,4.443908e+06,4.443908e+06,4.443908e+06,4.443908e+06,4.443908e+06,4.443908e+06,4.443908e+06,...,4.443908e+06,4.443908e+06,4.443908e+06,4.443908e+06,4.443908e+06,4.443908e+06,4.443908e+06,4.443908e+06,4.443908e+06,4.443908e+06
mean,2.331207e-01,1.107155e+00,1.304688e+02,6.576635e-01,2.261651e-01,1.367619e+00,4.761735e+01,5.051326e+02,9.224142e-01,5.436123e-01,...,4.511573e+00,2.384028e-02,7.921406e-03,1.154785e+03,3.653804e+00,6.065740e+02,4.726742e-01,1.765604e+03,9.419725e+01,2.474774e+00
std,5.851941e-01,1.715637e+00,1.694026e+02,1.142982e+00,5.962883e-01,2.660753e+00,2.745958e+01,6.275603e+02,1.542878e+00,7.104304e-01,...,3.051011e+01,1.672169e-01,9.262881e-02,1.183486e+03,2.405069e+00,7.397233e+02,3.073201e-01,2.183109e+03,6.637064e+00,3.864503e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.555000e+02,2.000000e+00,0.000000e+00,2.000000e-01,1.585000e+02,9.300000e+01,0.000000e+00
50%,0.000000e+00,0.000000e+00,8.418000e+01,0.000000e+00,0.000000e+00,0.000000e+00,4.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,6.866000e+02,3.000000e+00,0.000000e+00,4.583000e-01,7.897000e+02,9.500000e+01,1.000000e+00
75%,0.000000e+00,2.000000e+00,1.860000e+02,1.000000e+00,0.000000e+00,2.000000e+00,7.100000e+01,1.172000e+03,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.977000e+03,5.000000e+00,1.495000e+03,7.407000e-01,2.729000e+03,9.700000e+01,4.000000e+00
max,1.700000e+01,3.300000e+01,4.238000e+03,3.900000e+01,3.500000e+01,3.500000e+01,1.010000e+02,2.170000e+03,4.000000e+01,2.000000e+01,...,3.823000e+03,1.200000e+01,5.000000e+00,2.578000e+04,5.000000e+01,2.013000e+03,1.000000e+00,4.127010e+04,1.000000e+02,5.000000e+01


### Normalizing

In [18]:
df['killsNorm'] = df['kills']*((100-df['playersJoined'])/100 + 1)
df['damageDealtNorm'] = df['damageDealt']*((100-df['playersJoined'])/100 + 1)
df['maxPlaceNorm'] = df['maxPlace']*((100-df['playersJoined'])/100 + 1)
df['matchDurationNorm'] = df['matchDuration']*((100-df['playersJoined'])/100+1)
df['killPlace'] = df['killPlace']*((100 - df['playersJoined'])/100 + 1)

# Model 

In [19]:
columns = ['killsNorm', 'damageDealtNorm', 'matchDurationNorm','healsandboosts', 'DBNOs', 'killPlace','walkDistance', 'winPoints'
          ,'weaponsAcquired', 'killStreaks', 'longestKill', 'teamKills', 'maxPlace', 'assists', 'revives'
          ,'numGroups']

## With PCA

In [20]:
PCA_X = df.loc[:, columns].values
PCA_y = df.loc[:, ['winPlacePerc']].values

PCA_X = StandardScaler().fit_transform(PCA_X)

pca = PCA(n_components=7)
principal_components = pca.fit_transform(PCA_X)
principal_df = pd.DataFrame(data = principal_components, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4', 'principal component 5', '  principal component 6', 'principal component 7'])

pca_df = pd.concat([principal_df, df[['winPlacePerc']]], axis = 1)

PCA_X = pca_df[pca_df.columns[0:7]]
PCA_y = pca_df[pca_df.columns[7]]

train_X, val_X, train_y, val_y = train_test_split(PCA_X,PCA_y, test_size=0.2, random_state=1)
model = LinearRegression()
model.fit(train_X, train_y)
pred = model.predict(val_X)
print(np.sqrt(metrics.mean_squared_error(val_y, pred)))
print(model.score(val_X, val_y))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [21]:
PCA_X = pca_df[pca_df.columns[0:7]]
PCA_y = pca_df[pca_df.columns[7]]

train_X, val_X, train_y, val_y = train_test_split(PCA_X,PCA_y, test_size=0.2, random_state=1)
regressor = RandomForestRegressor(n_estimators = 50, random_state = 0, min_samples_leaf=3, max_features='sqrt')
regressor.fit(train_X, train_y)
predict = regressor.predict(val_X)
print(np.sqrt(metrics.mean_squared_error(val_y, predict)))
print(regressor.score(val_X, val_y))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## Without PCA

In [ ]:
X = df[columns]
y = df['winPlacePerc']

train_X, val_X, train_y, val_y = train_test_split(X,y, test_size=0.2, random_state=1)
model = LinearRegression()
model.fit(train_X, train_y)
pred = model.predict(val_X)
print(np.sqrt(metrics.mean_squared_error(val_y, pred)))
print(model.score(val_X, val_y))

In [ ]:
regressor = RandomForestRegressor(n_estimators = 50, random_state = 0, min_samples_leaf=3, max_features='sqrt')
regressor.fit(train_X, train_y)
predict = regressor.predict(val_X)
print(np.sqrt(metrics.mean_squared_error(val_y, predict)))
print(regressor.score(val_X, val_y))